Antes de empezar: un poco de documentación para enterder ligeramente cómo funciona HTML:
https://www.w3schools.com/html/html_intro.asp

# Web Scraping: Selenium

En este notebook veremos cómo automatizar la extracción de datos de una página web simulando la navegación. Para ello utilizaremos la librería Selenium:
https://selenium-python.readthedocs.io/

El software libre Selenium es un framework para realizar test automatizados de software a aplicaciones web. En principio, fue desarrollado para poner a prueba páginas y apps web, pero el WebDriver de Selenium también puede usarse con Python para realizar scraping. Si bien Selenium en sí no está escrito en Python, con este lenguaje de programación es posible acceder a las funciones del software.

A diferencia de Scrapy y de BeautifulSoup, Selenium no trabaja con el texto fuente en HTML de la web en cuestión, sino que carga la página en un navegador sin interfaz de usuario. El navegador interpreta entonces el código fuente de la página y crea, a partir de él, un Document Object Model (modelo de objetos de documento o DOM). Esta interfaz estandarizada permite poner a prueba las interacciones de los usuarios. De esta forma se consigue, por ejemplo, simular clics y rellenar formularios automáticamente. Los cambios en la web que resultan de dichas acciones se reflejan en el DOM. La estructura del proceso de web scraping con Selenium es la siguiente:

URL → Solicitud HTTP → HTML → Selenium → DOM

In [1]:
# Imports
import pandas as pd
from selenium import webdriver

Deberemos descargar un archivo Chrome driver para selenium en https://chromedriver.chromium.org/downloads, es importante que compruebes la versión de Chrome que utilizas y descargaues el driver correspondiente.
Una vez descargado crearemos una variable con la dirección completa del path al chromedriver 

El objeto driver es con el que trabajaremos a partir de ahora

In [2]:
# especificamos el path hasta nuestro driver recién descargado:
chrome_driver_path = '/Users/borjapuig/Downloads/chromedriver 2'
options  = webdriver.ChromeOptions()
#options.add_argument('headless') 
# headless nos permite no tener que ver la navegación en la ventana de Chrome

In [3]:
# Creamos el driver con el que nos vamos a manejar en la sesión de scrapeo:
driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)

In [4]:
# indicamos la URL de la página web a la que queremos acceder:
url = 'https://www.filmaffinity.com/es/main.html'
# el objeto driver nos va a permitir alterar el estado del la página
driver.get(url)

La página de Filmaffinity se ha abierto pero....¡Vaya! Nos hemos encontrado con un pop-up que nos pide aceptar cookies o algo po el estilo, si nos lo queremos saltar deberemos hacer lo siguiente:

1. Buscamos el botón
2. Hacemos click en el botón

Selenium nos permite buscar elementos por etiqueteas de html: https://selenium-python.readthedocs.io/locating-elements.html

CUIDADO! Los metodos de busqueda de elementos estan "duplicados". Tenemos *find_element_by_tag_name* y *find_elements_by_tag_name* el primero nos devolverá el primer elemento que encuentre (aunque haya más), el segundo nos devolverá una lista con todos los elementos que encuentre (aunque solo sea uno)

In [5]:
# Navegando en filmaffinity, en el pop-up:
elements_by_tag = driver.find_elements_by_tag_name('button')
elements_by_class_name = driver.find_elements_by_class_name('css-47sehv')
element_by_xpath = driver.find_element_by_xpath('/html/body/div[1]/div/div/div/div[2]/div/button[2]')

Una vez tenemos los elementos podemos hacer varias cosas con ellos

Podemos extraer todos los atributos que tenga

In [6]:
dir(element_by_xpath)
# obtenemos todos sus métodos y atributos:

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_execute',
 '_id',
 '_parent',
 '_upload',
 '_w3c',
 'clear',
 'click',
 'find_element',
 'find_element_by_class_name',
 'find_element_by_css_selector',
 'find_element_by_id',
 'find_element_by_link_text',
 'find_element_by_name',
 'find_element_by_partial_link_text',
 'find_element_by_tag_name',
 'find_element_by_xpath',
 'find_elements',
 'find_elements_by_class_name',
 'find_elements_by_css_selector',
 'find_elements_by_id',
 'find_elements_by_link_text',
 'find_elements_by_name',
 'find_elements_by_partial_link_text',
 'find_elements_by_tag_name',
 'find_elements_by_xpath',
 'get_attribute',
 'get_property',
 'id',
 

In [7]:
element_by_xpath.size

{'height': 40, 'width': 347}

In [7]:
element_by_xpath.get_attribute('size')

'large'

Podemos evaluar que tipo de elemento es (tag)

In [8]:
element_by_xpath.tag_name

'button'

Podemos sacar el valor que tiene (el texto)

In [9]:
element_by_xpath.text

'ACEPTO'

In [10]:
for i in range(0,len(elements_by_tag)):
    print(elements_by_tag[i].text)

socios
MÁS OPCIONES
ACEPTO


Incluso podemos guardar una imagen del elemento

In [11]:
type(element_by_xpath)
# Vemos que es tipo 'WebElement' y en la documentación podremos encontrar sus métodos

selenium.webdriver.remote.webelement.WebElement

In [12]:
# guardamos como 'mi_imagen.png' la imagen asociada al xpath
element_by_xpath.screenshot('mi_imagen.png')

True

Evaluamos que elementos hemos encontrado por el tag:

In [13]:
for index, element in enumerate(elements_by_tag):
    print('Elemento:', index)
    print('Texto del elemento',index, 'es', element.text)
    print('El tag del elemento',index, 'es', element.tag_name)
    element.screenshot('mi_imagen'+str(index)+'.png')

Elemento: 0
Texto del elemento 0 es socios
El tag del elemento 0 es button
Elemento: 1
Texto del elemento 1 es MÁS OPCIONES
El tag del elemento 1 es button
Elemento: 2
Texto del elemento 2 es ACEPTO
El tag del elemento 2 es button


Nos quedamos con el botón que nos interesa

In [14]:
boton_aceptar = elements_by_tag[2]

Si el elemento es interactivo podremos hacer más cosas además de las anteriores. Por ejemplo: hacer click

In [15]:
boton_aceptar.click()

Buscamos una película por título

In [16]:
from selenium.webdriver.common.keys import Keys

In [17]:
buscador = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div/div[2]/form/input[1]')

selenium.webdriver.remote.webelement.WebElement

In [18]:
buscador.send_keys('Robocop')

In [19]:
# una vez escrita la búsqueda deberíamos poder activarla:
buscador.send_keys(Keys.ENTER)

In [20]:
# volvemos a la página anterior
driver.back()

### Vamos a buscar todas las películas que se estrenan el próximo viernes

1. Cogemos los containers que hay en la zona lateral

In [21]:
menu_lateral = driver.find_element_by_id('lsmenu')
menu_lateral

<selenium.webdriver.remote.webelement.WebElement (session="3d6a1bdaea26b2c033d4be4e7b8983e1", element="fae14c5d-5982-48bf-8b8f-2dde20458a14")>

In [22]:
mis_secciones = menu_lateral.find_elements_by_tag_name('a')

2. Vemos con cuál nos tenemos que quedar

In [23]:
for a in mis_secciones:
    if a.text == 'Próximos estrenos':
        a.click()
        break

Accedemos al container central, en el que aparecen los estrenos por semana que queremos ver, exactamente igual que hemos hecho antes

In [24]:
cajon_central = driver.find_elements_by_id('main-wrapper-rdcat')

In [25]:
type(cajon_central)

list

In [26]:
for semana in cajon_central:
    print(semana.find_element_by_tag_name('div').text)
    print(semana.find_element_by_tag_name('div').get_attribute('id'))

21 de mayo de 2021
2021-05-21
28 de mayo de 2021
2021-05-28
3 de junio de 2021
2021-06-03
4 de junio de 2021
2021-06-04
11 de junio de 2021
2021-06-11
16 de junio de 2021
2021-06-16
18 de junio de 2021
2021-06-18
25 de junio de 2021
2021-06-25


In [27]:
for semana in cajon_central:
    fecha = semana.find_element_by_tag_name('div').get_attribute('id')
    if fecha == '2021-05-21':
        break

Buscamos cómo acceder a las películas

In [28]:
caratulas = semana.find_elements_by_class_name('mc-poster')
lista_pelis = []
for peli in caratulas:
    lista_pelis.append(peli.find_element_by_tag_name('a').get_attribute('href'))

In [29]:
lista_pelis

['https://www.filmaffinity.com/es/film637149.html',
 'https://www.filmaffinity.com/es/film478891.html',
 'https://www.filmaffinity.com/es/film713795.html',
 'https://www.filmaffinity.com/es/film112106.html',
 'https://www.filmaffinity.com/es/film329087.html',
 'https://www.filmaffinity.com/es/film950177.html',
 'https://www.filmaffinity.com/es/film388427.html',
 'https://www.filmaffinity.com/es/film132801.html',
 'https://www.filmaffinity.com/es/film759533.html',
 'https://www.filmaffinity.com/es/film407520.html',
 'https://www.filmaffinity.com/es/film452148.html',
 'https://www.filmaffinity.com/es/film158082.html',
 'https://www.filmaffinity.com/es/film108823.html']

Una vez tenemos todas las urls vamos a ver qué hacemos con cada una de ellas

In [30]:
# Accedemos a la página de la primera pelicula
driver.get(lista_pelis[0])

Vamos a ver el proceso que deberíamos hacer con cada una de las películas:

1. Sacamos toda la información que nos interesa

In [31]:
# titulo, nota, numero de votos y ficha técnica
titulo = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/h1/span').text
nota = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[1]').text
votos = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]/span').text
ficha = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]')

In [32]:
titulo

'Spiral: Saw'

2. Creamos una lista a partir de la ficha técnica

In [33]:
# Los nombres estan con tag  = 'dt' y los valores con 'dd'
ficha_names = []
ficha_values = []

for name in ficha.find_elements_by_tag_name('dt'):
    ficha_names.append(name.text)
for value in ficha.find_elements_by_tag_name('dd'):
    ficha_values.append(value.text)

In [34]:
ficha_values

['Spiral: From the Book of Saw',
 '2021',
 '93 min.',
 ' Estados Unidos',
 'Darren Lynn Bousman',
 'Josh Stolberg, Pete Goldfinger. Historia: Pete Goldfinger, Chris Rock, Josh Stolberg',
 'Charlie Clouser',
 'Jordan Oram',
 'Chris Rock, Samuel L. Jackson, Max Minghella, Marisol Nichols, Zoie Palmer, Nazneen Contractor, Dan Petronijevic, Morgan David Jones, Edie Inksetter, K.C. Collins, Josh Stolberg, Ali Johnson, Brian Cook, ver 6 más',
 'Serendipity Productions, Twisted Pictures, Burg/Koules Productions, Dahlstar. Distribuidora: Lionsgate',
 'Terror. Thriller | Policíaco. Crimen. Asesinos en serie',
 'Saw',
 'Trabajando a la sombra de un policía veterano (Samuel L. Jackson), el intrépido detective Ezekiel “Zeke” Banks (Chris Rock) y su compañero novato (Max Minghella) toman las riendas de una investigación relacionada con una serie de asesinatos que albergan ciertos paralelismos con el sangriento pasado de la ciudad. A medida que profundiza en el misterio, Zeke descubre que él mismo s

3. Creamos un dataframe con la info

In [35]:
columns = ['Titulo', 'Nota', 'Votos']
columns.extend(ficha_names)
len(columns)

16

In [36]:
values = [titulo, nota, votos]
values.extend(ficha_values)
len(values)

16

In [37]:
pd.DataFrame([values],columns=columns)

,Titulo,Nota,Votos,Título original,Año,Duración,País,Dirección,Guion,Música,Fotografía,Reparto,Productora,Género,Grupos,Sinopsis
0,Spiral: Saw,"5,1",35,Spiral: From the Book of Saw,2021,93 min.,Estados Unidos,Darren Lynn Bousman,"Josh Stolberg, Pete Goldfinger. Historia: Pete...",Charlie Clouser,Jordan Oram,"Chris Rock, Samuel L. Jackson, Max Minghella, ...","Serendipity Productions, Twisted Pictures, Bur...",Terror. Thriller | Policíaco. Crimen. Asesinos...,Saw,Trabajando a la sombra de un policía veterano ...


Ahora vamos a crear una función que nos haga todo esto para cada una de las películas:

In [38]:
def sacar_info(driver):
    
    titulo = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/h1/span').text
    # try/except lo dejamos para el final
    try:
        nota = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]').text
        votos = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]').text
    except:
        nota = None
        votos = None
    ficha = driver.find_element_by_xpath('/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]')
    
    return titulo, nota, votos, ficha

def sacar_ficha(ficha):
    
    ficha_names = []
    ficha_values = []

    for name in ficha.find_elements_by_tag_name('dt'):
        ficha_names.append(name.text)
    for value in ficha.find_elements_by_tag_name('dd'):
        ficha_values.append(value.text)
        
    return ficha_names, ficha_values

def montar_df(ficha_names, ficha_values, titulo, nota, votos):
    
    columns = ['Titulo', 'Nota', 'Votos']
    columns.extend(ficha_names)
    values = [titulo, nota, votos]
    values.extend(ficha_values)
    
    return pd.DataFrame([values], columns = columns)
    
def nueva_pelicula(driver):
    
    titulo, nota, votos, ficha = sacar_info(driver)
    ficha_names, ficha_values = sacar_ficha(ficha)
    df_peli = montar_df(ficha_names, ficha_values, titulo, nota, votos)
    
    return df_peli


## Modo Dios: moviendonos entre ventanas

Vamos a ver cómo nos podemos mover entre ventanas del navegador

Abrir nueva ventana:

In [39]:
driver.execute_script('window.open("");')

Movernos a otra ventana

In [40]:
driver.switch_to.window(driver.window_handles[0])

Cerrar ventana

In [41]:
driver.close()
# No se puede ejecutar más de una vez porque el driver sigue siendo la que se cerró, 
# hay que moverse de ventana y aí sí se puede cerrar

Una vez cerramos la ventana tenemos que indicarle a qué ventana tiene que ir

In [42]:
driver.switch_to.window(driver.window_handles[-1])

Sabiendo cómo podemos movernos por entre las ventanas y sabiendo cómo extraer de cada página toda la información que necesitamos vamos a crear nuestro dataframe:

In [43]:
# para abrir todos los links en lista_pelis
for link in lista_pelis:
    driver.execute_script('window.open("'+link+'");')
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(link)

In [44]:
# Creamos un dataframe con todas las pelis que se estrenan la próxima semana:
df_peliculas = pd.DataFrame()

for link in lista_pelis:
    driver.execute_script('window.open("");')
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(link)
    nueva_peli = nueva_pelicula(driver)
    df_peliculas = df_peliculas.append(nueva_peli)

In [45]:
df_peliculas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 0 to 0
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Titulo           13 non-null     object
 1   Nota             7 non-null      object
 2   Votos            7 non-null      object
 3   Título original  13 non-null     object
 4   Año              13 non-null     object
 5   Duración         12 non-null     object
 6   País             13 non-null     object
 7   Dirección        13 non-null     object
 8   Guion            13 non-null     object
 9   Música           10 non-null     object
 10  Fotografía       13 non-null     object
 11  Reparto          13 non-null     object
 12  Productora       13 non-null     object
 13  Género           13 non-null     object
 14  Grupos           2 non-null      object
 15  Sinopsis         13 non-null     object
 16                   3 non-null      object
dtypes: object(17)
memory usage: 1.8+ KB


In [46]:
df_peliculas

,Titulo,Nota,Votos,Título original,Año,Duración,País,Dirección,Guion,Música,Fotografía,Reparto,Productora,Género,Grupos,Sinopsis,
0,Spiral: Saw,"5,1\n35\nvotos",35\nvotos,Spiral: From the Book of Saw,2021,93 min.,Estados Unidos,Darren Lynn Bousman,"Josh Stolberg, Pete Goldfinger. Historia: Pete...",Charlie Clouser,Jordan Oram,"Chris Rock, Samuel L. Jackson, Max Minghella, ...","Serendipity Productions, Twisted Pictures, Bur...",Terror. Thriller | Policíaco. Crimen. Asesinos...,Saw,Trabajando a la sombra de un policía veterano ...,NaN
0,Poliamor para principiantes,None,None,Poliamor para principiantes,2021,97 min.,España,Fernando Colomo,"Fernando Colomo, Casandra Macías Gago, Marina ...",Fernando Furones,Ángel Iguacel,"Karra Elejalde, Toni Acosta, Quim Àvila Conde,...","Coproducción España-Francia; Morena Films, Ran...",Comedia. Romance | Comedia romántica,NaN,Satur (Karra Elejalde) y Tina (Toni Acosta) so...,NaN
0,Érase una vez en Queens,None,None,Boogie,2021,89 min.,Estados Unidos,Eddie Huang,Eddie Huang,"Ali Shaheed Muhammad, Adrian Younge",Brett Jutkiewicz,"Taylor Takahashi, Domenick Lombardozzi, Taylou...","Immersive Pictures, Wink Productions. Distribu...",Drama | Baloncesto. Inmigración,NaN,"En la Nueva York contemporánea, un joven fenóm...",NaN
0,First Cow,"6,9\n1.391\nvotos",1.391\nvotos,First Cow,2019,121 min.,Estados Unidos,Kelly Reichardt,"Jonathan Raymond, Kelly Reichardt. Novela: Jon...",William Tyler,Christopher Blauvelt,"John Magaro, Orion Lee, Toby Jones, Ewen Bremn...","Film Science, A24, IAC Films. Distribuidora: A24",Drama. Western | Siglo XIX. Amistad,NaN,Narra la historia de un cocinero (John Magaro)...,NaN
0,Mía y Moi,"6,0\n60\nvotos",60\nvotos,Mía y Moi,2021,NaN,España,Borja de la Vega,Borja de la Vega,NaN,Álvaro Ruiz,"Bruna Cusí, Ricardo Gómez, Eneko Sagardoy, Joe...","Paciencia y Baraja, Toned Media",Drama,NaN,"Tras perder a su madre, Mía y Moi se refugian ...",NaN
0,Pequeño país,"6,4\n28\nvotos",28\nvotos,Petit pays aka,2020,111 min.,Francia,Eric Barbier,Eric Barbier. Novela: Gaël Faye,Renaud Barbier,Antoine Sanier,"Jean-Paul Rouve, Djibril Vancoppenolle, Dayla ...","Coproducción Francia-Bélgica; Jerico, Super 8 ...",Drama,NaN,Gabriel tiene 10 años y vive en un cómodo veci...,
0,Sweat,"6,5\n34\nvotos",34\nvotos,Sweat,2020,100 min.,Polonia,Magnus von Horn,Magnus von Horn,Piotr Kurek,Michal Dymek,"Magdalena Kolesnik, Julian Swiezewski, Aleksan...","Coproducción Polonia-Suecia; Lava Films, Zentr...",Drama | Internet/Informática,NaN,Describe tres días en la vida de la motivadora...,NaN
0,Félix y el tesoro de Morgäa,None,None,Félix et le trésor de Morgäa aka,2021,90 min.,Canadá,Nicola Lemay,Marc Robitaille,NaN,Animación,"Animación, Gabriel Lessard, Guy Nadon, Marc La...",10e Ave,Animación. Aventuras | Adolescencia,NaN,Félix es un niño de 12 años que está convencid...,
0,El viaje de Chihiro,"8,1\n109.280\nvotos",109.280\nvotos,Sen to Chihiro no kamikakushi aka,2001,124 min.,Japón,Hayao Miyazaki,Hayao Miyazaki,Joe Hisaishi,"Animación, Atsushi Okui",Animación,"Studio Ghibli, Tokuma Shoten, Dentsu Inc. Dist...",Animación. Fantástico. Aventuras | Dragones. P...,Studio Ghibli (y obras relacionadas),Chihiro es una niña de diez años que viaja en ...,
0,Cascarrabias,None,None,Here Comes the Grump (Ahí viene Cascarrabias),2018,97 min.,México,Andrés Couturier,"Jim Hecht, Piers Clifton, Ricardo Hernández Du...",James Seymour Brett,Animación,Animación,Coproducción México-Estados Unidos; Anima Estu...,Animación. Aventuras. Comedia. Fantástico. Inf...,NaN,Pasar las vacaciones de verano en el increíble...,NaN


¡Tachán! Ya tenemos un dataframe con todas las películas que se van a estrenar el próximo viernes